# Using Skala with the Atomic Simulation Environment (ASE)

This tutorial provides a comprehensive overview of how to use the Skala neural network-based exchange-correlation functional with the [Atomic Simulation Environment (ASE)](https://wiki.fysik.dtu.dk/ase/). The Skala functional is available as an ASE calculator, enabling accurate and scalable density functional theory calculations on molecular and periodic systems.


In [1]:
import numpy as np
from ase.build import molecule
from ase.optimize import LBFGSLineSearch as Opt
from ase.units import Bohr, Hartree

from skala.ase import Skala

/home/giulialuise/miniconda3/envs/skala/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Basic Calculator Setup

We can create the `Skala` calculator and set options like the basis set and density fitting when constructing the calculator. The calculator supports several key parameters:

- `xc`: Exchange-correlation functional (default: "skala")
- `basis`: Basis set for the calculation (e.g., "def2-svp", "def2-tzvp")
- `with_density_fit`: Enable density fitting for faster calculations
- `with_dftd3`: Include DFT-D3 dispersion correction (default: True)
- `verbose`: Control output verbosity (0-4)

In [3]:
# Create a water molecule
atoms = molecule("H2O")

# Set up the Skala calculator with specific parameters
atoms.calc = Skala(xc="skala", basis="def2-svp", verbose=4)

# Display the calculator parameters
print("Calculator parameters:")
for key, value in atoms.calc.parameters.items():
    print(f"  {key}: {value}")

Calculator parameters:
  xc: skala
  basis: def2-svp
  with_density_fit: False
  with_newton: False
  with_dftd3: True
  charge: None
  multiplicity: None
  verbose: 4


## Energy Calculations

Calculations can be performed by requesting properties from the atoms object, which is connected to the calculator. For example, we can calculate the total energy of a water molecule. The energy is returned in eV (ASE's default energy unit):

In [4]:
# Calculate the total energy
energy_eV = atoms.get_potential_energy()  # eV
energy_Hartree = energy_eV / Hartree  # Convert to Hartree

print(f"Total energy: {energy_eV:.6f} eV")
print(f"Total energy: {energy_Hartree:.6f} Hartree")

System: uname_result(system='Linux', node='GCRAZGDL1498', release='6.8.0-1034-azure', version='#39~22.04.1-Ubuntu SMP Wed Aug 13 22:25:47 UTC 2025', machine='x86_64')  Threads 24
Python 3.13.7 | packaged by conda-forge | (main, Sep  3 2025, 14:30:35) [GCC 14.3.0]
numpy 2.3.3  scipy 1.16.2  h5py 3.14.0
Date: Tue Oct  7 15:43:37 2025
PySCF version 2.9.0
PySCF path  /home/giulialuise/miniconda3/envs/skala/lib/python3.13/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.119262000000 AA    0.000000000000   0.000000000000   0.225372517068 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.763239000000  -0.477047000000 

## Updating Calculator Parameters

To update the calculator parameters, we can use the `set` method. For example, we can activate density fitting to speed up the calculation and make the output less verbose. Note that changing certain parameters will reset the calculator and clear previous results.

In [5]:
# Update calculator settings
changed_params = atoms.calc.set(with_density_fit=True, verbose=0)
print(changed_params)
print(f"Changed parameters: {changed_params}")

# Show current parameters
print("\nCurrent calculator parameters:")
for key, value in atoms.calc.parameters.items():
    print(f"  {key}: {value}")

print(atoms.calc)

{'with_density_fit': True, 'verbose': 0}
Changed parameters: {'with_density_fit': True, 'verbose': 0}

Current calculator parameters:
  xc: skala
  basis: def2-svp
  with_density_fit: True
  with_newton: False
  with_dftd3: True
  charge: None
  multiplicity: None
  verbose: 0


## Force Calculations

We can continue to use the calculator for further calculations, such as computing the forces on the atoms. Since we changed the settings above, the calculator was reset and all previous results have been cleared. Requesting the forces will trigger a new calculation with the updated parameters.

In [6]:
# Calculate forces
forces = atoms.get_forces()  # eV/Å

print("Forces on atoms (eV/Å):")
for i, (symbol, force) in enumerate(zip(atoms.get_chemical_symbols(), forces)):
    print(
        f"  Atom {i+1} ({symbol}): [{force[0]:8.4f}, {force[1]:8.4f}, {force[2]:8.4f}]"
    )

# Calculate maximum force component
max_force = np.max(np.abs(forces))
print(f"\nMaximum force component: {max_force:.4f} eV/Å")

/home/giulialuise/.cache/huggingface/hub/models--microsoft--skala/snapshots/9eedaca41deeaebe197bec77d1898f777770d3ef/skala-1.0.fun
Downloaded in 0.10 seconds
Forces on atoms (eV/Å):
  Atom 1 (O): [  0.0000,   0.0000,  -0.5867]
  Atom 2 (H): [ -0.0000,  -0.6148,   0.2933]
  Atom 3 (H): [  0.0000,   0.6148,   0.2933]

Maximum force component: 0.6148 eV/Å


## Geometry Optimization

To use the calculator for geometry optimization or molecular dynamics simulations, you can use the `optimize` or `dynamics` modules from ASE. Here we'll perform a geometry optimization to find the minimum energy structure:

In [7]:
# Store initial geometry for comparison
initial_positions = atoms.positions.copy()
initial_energy = atoms.get_potential_energy()

# Set up and run geometry optimization
opt = Opt(atoms, trajectory="water_opt.traj")
opt.run(fmax=0.01)  # Optimize until forces are below 0.01 eV/Å

# Show optimization results
final_energy = atoms.get_potential_energy()
final_forces = atoms.get_forces()
max_final_force = np.max(np.abs(final_forces))

print(f"Optimization completed in {opt.get_number_of_steps()} steps")
print(f"Initial energy: {initial_energy:.6f} eV")
print(f"Final energy:   {final_energy:.6f} eV")
print(f"Energy change:  {final_energy - initial_energy:.6f} eV")
print(f"Maximum final force: {max_final_force:.4f} eV/Å")

                 Step     Time          Energy          fmax
LBFGSLineSearch:    0 15:43:56    -2076.458638        0.681230
LBFGSLineSearch:    1 15:44:02    -2076.466242        0.308524
LBFGSLineSearch:    2 15:44:13    -2076.467743        0.153213
LBFGSLineSearch:    3 15:44:26    -2076.468485        0.002282
Optimization completed in 3 steps
Initial energy: -2076.458638 eV
Final energy:   -2076.468485 eV
Energy change:  -0.009847 eV
Maximum final force: 0.0023 eV/Å


## Dipole Moment Calculations

The Skala calculator also supports dipole moment calculations, which are useful for understanding molecular polarity:

In [8]:
# Calculate dipole moment
dipole = atoms.get_dipole_moment()  # Debye

print(
    f"Dipole moment vector (Debye): [{dipole[0]:8.4f}, {dipole[1]:8.4f}, {dipole[2]:8.4f}]"
)
print(f"Dipole moment magnitude: {np.linalg.norm(dipole):.4f} Debye")

Dipole moment vector (Debye): [ -0.0000,   0.0000,  -0.4200]
Dipole moment magnitude: 0.4200 Debye


## Working with Different Molecules and Basis Sets

Let's explore how to work with different molecular systems and basis sets. Here we'll calculate properties for methane (CH₄) using a larger basis set:

In [9]:
# Create a methane molecule
atoms = molecule("CH4")

# Set up calculator with a larger basis set and density fitting
atoms.calc = Skala(
    xc="skala",
    basis="def2-tzvp",  # Triple-zeta basis set
    with_density_fit=True,  # Enable density fitting for efficiency
    with_dftd3=True,  # Include dispersion correction
    verbose=1,
)

print("Methane molecule:")
print(f"Chemical formula: {atoms.get_chemical_formula()}")
print(f"Number of atoms: {len(atoms)}")

# Calculate properties
energy = atoms.get_potential_energy()
forces = atoms.get_forces()
max_force = np.max(np.abs(forces))

print(f"\nCalculation results:")
print(f"Total energy: {energy:.6f} eV")
print(f"Maximum force: {max_force:.6f} eV/Å")

Methane molecule:
Chemical formula: CH4
Number of atoms: 5
/home/giulialuise/.cache/huggingface/hub/models--microsoft--skala/snapshots/9eedaca41deeaebe197bec77d1898f777770d3ef/skala-1.0.fun
Downloaded in 0.11 seconds

Calculation results:
Total energy: -1102.070344 eV
Maximum force: 0.351115 eV/Å


## Summary

This tutorial covered the essential features of the Skala ASE calculator:

- Setting up the calculator with various parameters
- Calculating energies, forces, and dipole moments
- Updating calculator parameters dynamically
- Performing geometry optimizations
- Working with different molecules and basis sets